In [1]:
import time
import pandas as pd
import psycopg2
import datetime
from tqdm import trange
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import seaborn as sns
import math
# %matplotlib inline 
import platform
import numpy as np
from matplotlib import font_manager, rc # plt.rcParams['axes.unicode_minus'] = False 
if platform.system() == 'Darwin': rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': 
    path = "c:/Windows/Fonts/malgun.ttf" 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name) 
else: 
    print('Unknown system... sorry~~~~')
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
from tqdm import tqdm
from tqdm import trange
from datetime import datetime, timedelta


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [3]:
# 시계열
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [4]:
    # row 생략 없이 출력
pd.set_option('display.max_rows', 1000)
# col 생략 없이 출력
pd.set_option('display.max_columns', 1000)
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
pd.options.display.float_format = '{:.5f}'.format

In [5]:
# 전체 매장 입출판재
total_shop_stock = pd.read_csv("total_shop_stock.csv")
total_shop_stock = total_shop_stock.iloc[ :,1:]

# 매장정보
shop_info_pd = pd.read_csv("shop_info_pd.csv")
shop_info_pd = shop_info_pd.iloc[ :,1:]

# bucket_dwalker 관련 매장입출판재(2019)
bucket_dwalker = pd.read_csv('bucket_dwalker.csv')
bucket_dwalker = bucket_dwalker.iloc[ :,1:]

# bucket_dwalker_v2 관련 매장입출판재
bucket_dwalker_v2 = pd.read_csv('bucket_dwalker_v2.csv')
bucket_dwalker_v2 = bucket_dwalker_v2.iloc[ :,1:]

# 아울렛 상설을 제외한 매장키 
shop_list_pd = pd.read_csv('shop_list.csv')
shop_list_pd = shop_list_pd.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 기준)
shop_weeks_buket_df = pd.read_csv('shop_weeks_buket.csv')
shop_weeks_buket_df = shop_weeks_buket_df.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 v2 기준)
shop_weeks_buket_v2_df = pd.read_csv('shop_weeks_buket_v2.csv')
shop_weeks_buket_v2_df = shop_weeks_buket_v2_df.iloc[ :,1:]

# 전체 매장 비어있는것 채워넣은 입출판재 합쳐 놓은 것
shop_total_sales = pd.read_csv("shop_total_sales.csv")
shop_total_sales = shop_total_sales.iloc[ :,1:]

In [91]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

In [114]:
# 2019년 버킷디워커 물류 수량
start = time.time()

bk_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270') AND weekkey BETWEEN To_date('2018-12-31', 'YYYY-MM-DD') AND To_date('2019-12-31', 'YYYY-MM-DD')


'''


cur.execute(bk_stock_scsweek)
bk_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

bk_stock_scsweek_pd = pd.DataFrame(bk_stock_scsweek)
bk_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 0.1534135341644287


In [117]:
bk_stock_scsweek_pd.to_csv('bk_stock_scsweek_pd.csv')

In [94]:
# 2019년 버킷디워커 물류 수량
start = time.time()

v2_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270') AND weekkey BETWEEN To_date('2019-11-18', 'YYYY-MM-DD') AND To_date('2020-12-31', 'YYYY-MM-DD')


'''


cur.execute(v2_stock_scsweek)
v2_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

v2_stock_scsweek_pd = pd.DataFrame(v2_stock_scsweek)
v2_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 0.11534643173217773


In [116]:
v2_stock_scsweek_pd.to_csv('v2_stock_scsweek_pd.csv')

In [95]:
bk_stock_scsweek_pd

,weekkey,scskey,week_accept_qty,week_wsl_takeout_qty,wms_stock_qty,stock_qty,domestic_sale_qty
0,2019-01-07,X19SDXSH09911BE235,0,0,1,161,19
1,2019-01-14,X19SDXSH09911BE235,150,0,76,132,198
2,2019-01-21,X19SDXSH09911BE235,10,0,6,14,326
3,2019-01-28,X19SDXSH09911BE235,21,1,5,-39,399
4,2019-02-18,X19SDXSH09911BE235,520,1,82,274,865
5,2019-07-22,X19SDXSH09911BE235,0,4,454,1934,4674
6,2019-08-05,X19SDXSH09911BE235,0,0,384,1840,4768
7,2019-09-16,X19SDXSH09911BE235,0,1,118,1260,5342
8,2019-09-23,X19SDXSH09911BE235,0,1,135,1130,5471
9,2019-10-07,X19SDXSH09911BE235,0,0,74,898,5703


In [6]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

# mae_scorer = make_scorer(mae)
# mae_scorer

In [7]:
def make_ewm_arr(data_frame, span_num):
    arr_ewm = np.array([])
#     for i in data_frame.store_id.unique():
    df_set = data_frame
        # 여기에서 지정하는 span값은 위 수식에서 N에 해당한다.
    arr_ewm = np.concatenate((arr_ewm, df_set.week_sale_qty.ewm(span=span_num).mean().values) )
    
    return arr_ewm

# 재고가 0 인 지점

## 2020년 버킷디워커 v2

In [8]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='백화점']

In [12]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey'] != 'X10047']

In [13]:
v2_shopkey_unique =  shop_weeks_buket_v2_df_dep.shopkey.unique()
len(v2_shopkey_unique)

66

In [14]:
v2_scskey_unique =  shop_weeks_buket_v2_df_dep.scskey.unique()
len(v2_scskey_unique)

6

In [38]:
v2_shop_qty = pd.DataFrame()

for i in v2_shopkey_unique:
    uniqe_shop = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
    for j in v2_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        v2_shop_qty = pd.concat([v2_shop_qty, uniqe_shop_scskey], axis=0)

<ipython-input-38-7d2f0dd62d4a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-38-7d2f0dd62d4a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [39]:
v2_shop_qty

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
0,2019-11-18,1,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.00000,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
1,2019-11-25,0,1,0,0,1,X10056,0.00000,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.00000,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
2,2019-12-02,2,3,1,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.69231,XH0247,일반매장,백화점특정,백화점,경남,0.00000,1.00000
3,2019-12-09,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.22500,XH0247,일반매장,백화점특정,백화점,경남,0.69231,2.00000
4,2019-12-16,0,3,0,1,2,X10056,0.11111,X20SDXSHA5011OW230,갤러리아진주,디스커버리 갤러리아 진주점,0.07438,XH0247,일반매장,백화점특정,백화점,경남,0.22500,2.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50499,2020-11-30,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.11709,XH0237,일반매장,백화점특정,백화점,경남,0.35127,2.00000
50500,2020-12-07,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.03903,XH0237,일반매장,백화점특정,백화점,경남,0.11709,2.00000
50501,2020-12-14,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.01301,XH0237,일반매장,백화점특정,백화점,경남,0.03903,2.00000
50502,2020-12-21,0,52,0,44,8,X10038,1.00000,X20SDXSHA5011OW270,롯데창원,디스커버리 롯데 창원점,0.00434,XH0237,일반매장,백화점특정,백화점,경남,0.01301,2.00000


In [40]:
v2_zero = v2_shop_qty[(v2_shop_qty['stock_qty']==0) & (v2_shop_qty['takeout_qty']>0)]

In [41]:
len(v2_zero)

240

In [42]:
len(v2_zero[v2_zero['weekkey'] < '2020-11-01'])

205

In [46]:
# 재고에서 0 이 아닌 부분
len(v2_zero[v2_zero.qty - v2_zero.week_sale_qty>0])

78

In [47]:
# 재고에서 0 인 부분 
len(v2_zero[v2_zero.qty - v2_zero.week_sale_qty<=0])

162

In [105]:
v2_zero.groupby(['weekkey']).count().qty

weekkey
2019-11-18    19
2019-11-25    23
2019-12-02    19
2019-12-09    19
2019-12-16    13
2019-12-23    15
2019-12-30    11
2020-01-06    11
2020-01-13     6
2020-01-20     3
2020-01-27     6
2020-02-03     2
2020-02-10    12
2020-02-17     2
2020-03-02     8
2020-03-09     4
2020-03-16     5
2020-03-23     3
2020-04-06     3
2020-04-13     1
2020-05-25    14
2020-07-06     1
2020-07-27     1
2020-09-28     1
2020-10-19     3
2020-11-02     1
2020-11-09     1
2020-11-30     2
2020-12-07     2
2020-12-14     4
2020-12-21    15
2020-12-28    10
Name: qty, dtype: int64

In [106]:
shop_weeks_buket_v2_df_dep.groupby(['weekkey']).sum().stock_qty

weekkey
2019-11-18     634
2019-11-25     692
2019-12-02     780
2019-12-09     818
2019-12-16     893
2019-12-23     859
2019-12-30     946
2020-01-06     937
2020-01-13    1146
2020-01-20    1400
2020-01-27    1459
2020-02-03    1528
2020-02-10    1439
2020-02-17    1646
2020-02-24    1688
2020-03-02    1585
2020-03-09    1554
2020-03-16    1620
2020-03-23    1699
2020-03-30    1755
2020-04-06    1770
2020-04-13    1853
2020-04-20    1908
2020-04-27    1814
2020-05-04    1972
2020-05-11    1992
2020-05-18    2018
2020-05-25    1921
2020-06-01    2022
2020-06-08    2051
2020-06-15    2062
2020-06-22    2080
2020-06-29    2099
2020-07-06    2104
2020-07-13    2101
2020-07-20    2108
2020-07-27    2121
2020-08-03    2142
2020-08-10    2123
2020-08-17    2157
2020-08-24    2171
2020-08-31    2156
2020-09-07    2132
2020-09-14    2094
2020-09-21    2089
2020-09-28    1923
2020-10-05    2015
2020-10-12    2059
2020-10-19    2049
2020-10-26    2043
2020-11-02    2051
2020-11-09    2046
2020

In [107]:
v2_stock_scsweek_pd.groupby(['weekkey']).sum().wms_stock_qty

weekkey
2019-11-18     3224
2019-11-25     3350
2019-12-02     2611
2019-12-09     1941
2019-12-16     1087
2019-12-23      191
2019-12-30     1548
2020-01-06     1476
2020-01-13     5949
2020-01-20     5200
2020-01-27     4438
2020-02-03     3321
2020-02-10     1793
2020-02-17     1822
2020-02-24     1554
2020-03-02     1634
2020-03-09     3839
2020-03-16     7095
2020-03-23     7408
2020-03-30     7073
2020-04-06     6371
2020-04-13     8698
2020-04-20    10118
2020-04-27     9511
2020-05-04    11392
2020-05-11    12355
2020-05-18    11815
2020-05-25    11428
2020-06-01    10796
2020-06-08    10410
2020-06-15    10088
2020-06-22    10079
2020-06-29    10187
2020-07-06    11311
2020-07-13    14779
2020-07-20    14512
2020-07-27    12992
2020-08-03    12713
2020-08-10    12467
2020-08-17    12191
2020-08-24    11962
2020-08-31    11249
2020-09-07    10951
2020-09-14    10678
2020-09-21    10217
2020-09-28     9928
2020-10-05     9162
2020-10-12     8480
2020-10-19     7986
2020-10-26  

In [108]:
v2_zero.groupby(['weekkey','scskey']).count().qty.reset_index()

,weekkey,scskey,qty
0,2019-11-18,X20SDXSHA5011OW230,3
1,2019-11-18,X20SDXSHA5011OW235,5
2,2019-11-18,X20SDXSHA5011OW240,2
3,2019-11-18,X20SDXSHA5011OW260,4
4,2019-11-18,X20SDXSHA5011OW265,4
5,2019-11-18,X20SDXSHA5011OW270,1
6,2019-11-25,X20SDXSHA5011OW230,1
7,2019-11-25,X20SDXSHA5011OW235,7
8,2019-11-25,X20SDXSHA5011OW240,3
9,2019-11-25,X20SDXSHA5011OW260,5


In [109]:
shop_weeks_buket_v2_df_dep.groupby(['weekkey','scskey']).sum().stock_qty.reset_index()

,weekkey,scskey,stock_qty
0,2019-11-18,X20SDXSHA5011OW230,108
1,2019-11-18,X20SDXSHA5011OW235,102
2,2019-11-18,X20SDXSHA5011OW240,101
3,2019-11-18,X20SDXSHA5011OW260,105
4,2019-11-18,X20SDXSHA5011OW265,108
5,2019-11-18,X20SDXSHA5011OW270,110
6,2019-11-25,X20SDXSHA5011OW230,114
7,2019-11-25,X20SDXSHA5011OW235,111
8,2019-11-25,X20SDXSHA5011OW240,123
9,2019-11-25,X20SDXSHA5011OW260,106


In [110]:
v2_stock_scsweek_pd.groupby(['weekkey','scskey']).sum().wms_stock_qty.reset_index()

,weekkey,scskey,wms_stock_qty
0,2019-11-18,X20SDXSHA5011OW230,404
1,2019-11-18,X20SDXSHA5011OW235,551
2,2019-11-18,X20SDXSHA5011OW240,911
3,2019-11-18,X20SDXSHA5011OW260,424
4,2019-11-18,X20SDXSHA5011OW265,423
5,2019-11-18,X20SDXSHA5011OW270,511
6,2019-11-25,X20SDXSHA5011OW230,394
7,2019-11-25,X20SDXSHA5011OW235,530
8,2019-11-25,X20SDXSHA5011OW240,885
9,2019-11-25,X20SDXSHA5011OW260,446


## 2019년 버킷디워커

In [50]:
shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='백화점']

In [51]:
shop_weeks_buket_df_dep

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2018-12-31,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
1,2019-01-07,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
2,2019-01-14,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
3,2019-01-21,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
4,2019-01-28,0,0,0,0,0,X10056,0.00000,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45363,2019-12-02,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45364,2019-12-09,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45365,2019-12-16,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남
45366,2019-12-23,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,XH0237,일반매장,백화점특정,백화점,경남


In [52]:
bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
len(bk_shopkey_unique)

67

In [65]:
bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
len(bk_scskey_unique)

6

In [60]:
shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

In [66]:
bk_shop_qty = pd.DataFrame()

for i in bk_shopkey_unique:
    uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
    for j in bk_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

<ipython-input-66-d32067fabf12>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-66-d32067fabf12>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [67]:
bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]

In [68]:
bk_zero

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
6,2019-02-11,1,1,0,1,0,X10056,0.08333,X19SDXSH09911BE230,갤러리아진주,디스커버리 갤러리아 진주점,0.22232,XH0247,일반매장,백화점특정,백화점,경남,0.66758,2.00000
15162,2019-01-28,-1,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.29752,XH0247,일반매장,백화점특정,백화점,경남,0.90000,2.00000
15163,2019-02-04,0,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.09890,XH0247,일반매장,백화점특정,백화점,경남,0.29752,2.00000
15164,2019-02-11,0,2,0,2,0,X10056,0.10526,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.03294,XH0247,일반매장,백화점특정,백화점,경남,0.09890,2.00000
15166,2019-02-25,0,3,1,3,0,X10056,0.15789,X19SDXSH09911BE240,갤러리아진주,디스커버리 갤러리아 진주점,0.67036,XH0247,일반매장,백화점특정,백화점,경남,0.01098,2.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45363,2019-12-02,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00007,XH0237,일반매장,백화점특정,백화점,경남,0.00020,2.00000
45364,2019-12-09,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00002,XH0237,일반매장,백화점특정,백화점,경남,0.00007,2.00000
45365,2019-12-16,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00001,XH0237,일반매장,백화점특정,백화점,경남,0.00002,2.00000
45366,2019-12-23,0,21,0,21,0,X10038,1.00000,X19SDXSH09911BE270,롯데창원,디스커버리 롯데 창원점,0.00000,XH0237,일반매장,백화점특정,백화점,경남,0.00001,2.00000


In [70]:
len(bk_zero)

2225

In [71]:
# 재고에서 0 이 지워진 부분 
len(bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0])

1863

In [72]:
# 아직 재고가 0 이하인 부분
len(bk_zero[bk_zero.qty - bk_zero.week_sale_qty<=0])

362

In [111]:
# zero 인 지점
bk_zero.groupby(['weekkey','scskey']).count().qty.reset_index()

,weekkey,scskey,qty
0,2018-12-31,X19SDXSH09911BE235,1
1,2018-12-31,X19SDXSH09911BE240,1
2,2018-12-31,X19SDXSH09911BE260,2
3,2018-12-31,X19SDXSH09911BE270,1
4,2019-01-07,X19SDXSH09911BE230,1
5,2019-01-07,X19SDXSH09911BE235,3
6,2019-01-07,X19SDXSH09911BE240,2
7,2019-01-07,X19SDXSH09911BE260,6
8,2019-01-07,X19SDXSH09911BE265,2
9,2019-01-07,X19SDXSH09911BE270,4


In [112]:
# stock_qty 재고 개수
shop_weeks_buket_df_dep.groupby(['weekkey','scskey']).sum().stock_qty.reset_index()

,weekkey,scskey,stock_qty
0,2018-12-31,X19SDXSH09911BE230,15
1,2018-12-31,X19SDXSH09911BE235,63
2,2018-12-31,X19SDXSH09911BE240,79
3,2018-12-31,X19SDXSH09911BE260,62
4,2018-12-31,X19SDXSH09911BE265,17
5,2018-12-31,X19SDXSH09911BE270,63
6,2019-01-07,X19SDXSH09911BE230,12
7,2019-01-07,X19SDXSH09911BE235,59
8,2019-01-07,X19SDXSH09911BE240,74
9,2019-01-07,X19SDXSH09911BE260,58


In [115]:
bk_stock_scsweek_pd.groupby(['weekkey','scskey']).sum().wms_stock_qty.reset_index()

,weekkey,scskey,wms_stock_qty
0,2018-12-31,X19SDXSH09911BE230,0
1,2018-12-31,X19SDXSH09911BE235,1
2,2018-12-31,X19SDXSH09911BE240,1
3,2018-12-31,X19SDXSH09911BE260,1
4,2018-12-31,X19SDXSH09911BE265,0
5,2018-12-31,X19SDXSH09911BE270,1
6,2019-01-07,X19SDXSH09911BE230,0
7,2019-01-07,X19SDXSH09911BE235,1
8,2019-01-07,X19SDXSH09911BE240,1
9,2019-01-07,X19SDXSH09911BE260,1
